In [ ]:
import pandas as pd

# Wczytanie pliku CSV do DataFrame
df = pd.read_csv('platnosci karta.csv', encoding='utf-8', sep=';')

# Wyświetlenie informacji o DataFrame
print("Informacje o DataFrame:")
df.info()

# Wyświetlenie pierwszych 5 wierszy DataFrame
print("\nPierwsze 5 wierszy DataFrame:")
print(df.head())


In [ ]:
import pandas as pd
import re

def parse_amount(amt_str):
    """
    Przetwarza łańcuch z kwotą:
      - Usuwa spacje.
      - Jeśli łańcuch zawiera zarówno kropki, jak i przecinki, to zakładamy, 
        że kropki są separatorami tysięcy, więc je usuwamy, a przecinek zamieniamy na kropkę.
      - W przeciwnym wypadku zamieniamy przecinek na kropkę.
      Zwraca wartość float.
    """
    amt_str = amt_str.replace(' ', '').strip()
    if '.' in amt_str and ',' in amt_str:
        amt_str = amt_str.replace('.', '')
        amt_str = amt_str.replace(',', '.')
    else:
        amt_str = amt_str.replace(',', '.')
    try:
        return float(amt_str)
    except Exception:
        return 0.0

def parse_rozbicie(row):
    r"""
    Dla wiersza:
      1. Jeśli cała zawartość kolumny 'rozbicie' pasuje do wzorca pojedynczej rezerwacji 
         (tj. ewentualne spacje + 1\d{6} + ewentualne spacje),
         przyjmujemy, że jest tylko jedna rezerwacja – wtedy zwracamy ten numer,
         a kwota rezerwacji = kwota z kolumny 'kwota'.
      2. W przeciwnym wypadku:
         Próbujemy kolejno następujące schematy, gdzie w wyrażeniach regularnych:
            - (?P<res>...) wychwytuje numer rezerwacji (format: 1\d{6})
            - (?P<amt>...) wychwytuje kwotę (format: liczba z przecinkiem lub kropką)
         Schematy:
           a) rezerwacja-kwota: r'(?P<res>1\d{6})\s*-\s*(?P<amt>\d+[.,]\d{1,2})'
           b) kwota-rezerwacja: r'(?P<amt>\d+[.,]\d{1,2})\s*-\s*(?P<res>1\d{6})'
           c) rezerwacja(kwota): r'(?P<res>1\d{6})\s*\(\s*(?P<amt>\d+[.,]\d{1,2})\s*\)'
           d) rezerwacja kwota: r'(?P<res>1\d{6})\s+(?P<amt>\d+[.,]\d{1,2})'
           e) kwota rezerwacja: r'(?P<amt>\d+[.,]\d{1,2})\s+(?P<res>1\d{6})'
         Gdy któryś ze schematów zwróci dopasowania, zbieramy z niego pary (numer, kwota).
         
      Jeśli żaden schemat nie przyniesie wyniku, wyszukujemy same numery rezerwacji i przypisujemy im kwotę 0.0.
      
      Zwraca:
         - sum_rezerwacje: sumę wyekstrahowanych kwot (float)
         - rozbicie_norm: znormalizowany ciąg znakowy; dla jednej rezerwacji – sam numer,
           dla wielu – "numer1-kwota1; numer2-kwota2; ..." (kwoty sformatowane do dwóch miejsc po przecinku z przecinkiem jako separatorem)
    """
    text = row['rozbicie']
    # replace every \d+.\d{3,} with \d+\d{3,}. this dot is not a decimal separator, but a thousand separator and we dont want i
    text = re.sub(r'(\d+)\.(\d{3,})', r'\1\2', text)
    # Konwersja głównej kwoty z kolumny 'kwota'
    overall_kwota = parse_amount(row['kwota'])
    
    # 1. Sprawdzenie, czy całość to pojedyncza rezerwacja
    if re.fullmatch(r'\s*1\d{6}\s*', text):
        res_num = re.search(r'1\d{6}', text).group()
        return pd.Series({'sum_rezerwacje': overall_kwota, 'rozbicie_norm': res_num})

    # 2. Próbujemy rozpoznać pary według zadanych schematów
    patterns = [
        r'(?P<res>1\d{6})\s*-\s*(?P<amt>\d+[.,]\d{1,2})',   # rezerwacja-kwota
        r'(?P<amt>\d+[.,]\d{1,2})\s*-\s*(?P<res>1\d{6})',   # kwota-rezerwacja
        r'(?P<res>1\d{6})\s*\(\s*(?P<amt>\d+[.,]\d{1,2})\s*\)',  # rezerwacja(kwota)
        r'(?P<res>1\d{6})\s+(?P<amt>\d+[.,]\d{1,2})',        # rezerwacja kwota
        r'(?P<amt>\d+[.,]\d{1,2})\s+(?P<res>1\d{6})'         # kwota rezerwacja
    ]
    
    pairs = []
    # Przechodzimy kolejno po schematach – gdy pierwszy schemat da dopasowania, korzystamy z jego wyników.
    for pattern in patterns:
        matches = list(re.finditer(pattern, text))
        if matches:
            for m in matches:
                res = m.group('res')
                amt = parse_amount(m.group('amt'))
                pairs.append((res, amt))
            break

    # Jeśli żaden schemat nie zadziałał, wyszukujemy same numery rezerwacji i przypisujemy im kwotę 0.0
    if not pairs:
        reservations = re.findall(r'1\d{6}', text)
        for res in reservations:
            pairs.append((res, 0.0))
    
    total = sum(amt for _, amt in pairs)
    
    # Budujemy znormalizowany ciąg znakowy:
    if len(pairs) == 1:
        norm_str = pairs[0][0]
    else:
        norm_str = "; ".join(f"{r}-{format(a, '.2f').replace('.',',')}" for r, a in pairs)
    
    return pd.Series({'sum_rezerwacje': total, 'rozbicie_norm': norm_str})

# Przykładowe użycie:
# Zakładamy, że DataFrame df został już wczytany
df[['sum_rezerwacje', 'rozbicie_norm']] = df.apply(parse_rozbicie, axis=1)

# Dla porównania – konwersja kolumny 'kwota' na float z uwzględnieniem formatowania
df['kwota_num'] = df['kwota'].apply(lambda x: parse_amount(x) if x else 0.0)
# Obliczamy różnicę między oryginalną kwotą a sumą wyekstrahowanych kwot rezerwacji
df['roznica'] = df['kwota_num'] - df['sum_rezerwacje']

# Wyświetlamy kilka pierwszych wierszy dla weryfikacji
display_cols = ['kwota', 'kwota_num', 'sum_rezerwacje', 'roznica', 'rozbicie', 'rozbicie_norm']
print(df[display_cols].head())


In [3]:
# save to xlsx
df.to_excel('platnosci_karta_parsed.xlsx', index=False)